In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    model_name: str
    params_classes: int

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            model_name=self.params.MODEL_NAME,
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [13]:
import torch
import torch.nn as nn
import torchvision.models as models
from pathlib import Path



class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        print(f"[Init] PrepareBaseModel initialized with model: {self.config.model_name}")

    def get_base_model(self) -> nn.Module:
        """Load EfficientNet backbone based on config"""
        print("[Step 1] Loading base model...")
        if self.config.model_name == "efficientnet_b0":
            model = models.efficientnet_b0(weights=self.config.params_weights)
        elif self.config.model_name == "efficientnet_b1":
            model = models.efficientnet_b1(weights=self.config.params_weights)
        else:
            raise ValueError(f"Model {self.config.model_name} not supported")

        # Freeze layers if include_top is False
        if not self.config.params_include_top:
            print("[Step 2] Freezing base model layers...")
            for param in model.parameters():
                param.requires_grad = False

        # Replace classifier head
        print("[Step 3] Replacing classifier head...")
        in_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(512, self.config.params_classes)
        )

        print("[Done] Base model created.")
        return model

    def save_model(self, model: nn.Module, path: Path):
        """Save model weights"""
        torch.save(model.state_dict(), path)
        print(f"[Saved] Model saved at: {path}")

    def prepare(self):
        """Prepare and save the base model"""
        print("[Pipeline] Preparing base model...")
        model = self.get_base_model()
        self.save_model(model, self.config.base_model_path)
        print("[Pipeline Done] Base model prepared and saved.")
        return model

    def update_base_model(self, num_layers_to_unfreeze: int = 50):
        """
        Load the base model, unfreeze last N layers + classifier for fine-tuning,
        then save it as updated base model.
        """
        print("[Pipeline] Updating base model...")
        model = self.get_base_model()
        model.load_state_dict(torch.load(self.config.base_model_path))
        print("[Step 1] Loaded base model weights.")

        # Always unfreeze classifier
        print("[Step 2] Unfreezing classifier...")
        for param in model.classifier.parameters():
            param.requires_grad = True

        # Unfreeze last few backbone layers
        total_layers = len(list(model.features))
        layers_to_unfreeze = min(num_layers_to_unfreeze, total_layers)
        print(f"[Step 3] Unfreezing last {layers_to_unfreeze} backbone layers...")

        for i, child in enumerate(reversed(list(model.features.children()))):
            if i < layers_to_unfreeze:
                for param in child.parameters():
                    param.requires_grad = True

        # Save updated model
        self.save_model(model, self.config.updated_base_model_path)
        print("[Pipeline Done] Updated base model ready for fine-tuning.")

        return model


In [ ]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(prepare_base_model_config)

    # Step 1: Prepare base model
    base_model = prepare_base_model.prepare()

    # Step 2: Update (fine-tune) base model
    updated_model = prepare_base_model.update_base_model(num_layers_to_unfreeze=30)
except Exception as e:
    raise e

[2025-09-27 01:21:02,965: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-27 01:21:02,967: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-27 01:21:02,969: INFO: common: created directory at: artifacts]
[2025-09-27 01:21:02,971: INFO: common: created directory at: artifacts/prepare_base_model]
[Init] PrepareBaseModel initialized with model: efficientnet_b0
[Pipeline] Preparing base model...
[Step 1] Loading base model...
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\el papi/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:07<00:00, 2.79MB/s]


[Step 2] Freezing base model layers...
[Step 3] Replacing classifier head...
[Done] Base model created.
[Saved] Model saved at: artifacts\prepare_base_model\base_model.pth
[Pipeline Done] Base model prepared and saved.
[Pipeline] Updating base model...
[Step 1] Loading base model...
[Step 2] Freezing base model layers...
[Step 3] Replacing classifier head...
[Done] Base model created.
[Step 1] Loaded base model weights.
[Step 2] Unfreezing classifier...
[Step 3] Unfreezing last 9 backbone layers...
[Saved] Model saved at: artifacts\prepare_base_model\base_model_updated.pth
[Pipeline Done] Updated base model ready for fine-tuning.
